In [ ]:
import pandas as pd
import spacy

In [ ]:
df =pd.read_csv("calendar.csv.gz",compression="gzip")

In [ ]:
df.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,109,2022-12-07,f,$115.00,$115.00,30.0,730.0
1,109,2022-12-08,f,$115.00,$115.00,30.0,730.0
2,109,2022-12-09,f,$115.00,$115.00,30.0,730.0
3,109,2022-12-10,f,$115.00,$115.00,30.0,730.0
4,109,2022-12-11,f,$115.00,$115.00,30.0,730.0


In [ ]:
df_review =pd.read_csv("reviews.csv.gz",compression="gzip")

In [ ]:
df_review.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...
2,206662,780773,2011-12-12,1465676,Orlando,"Kevin is great Host, the apartment is clean an..."
3,206662,791960,2011-12-18,1465676,Orlando,"Excellent Host and excellent place, clean and ..."
4,206662,932852,2012-02-20,1559443,Nadine,I had a wonderful stay here and felt at home. ...


In [ ]:
df_listing =pd.read_csv("listings.csv.gz",compression ="gzip")

In [ ]:
df_listing.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,65467,https://www.airbnb.com/rooms/65467,20221206172243,2022-12-07,city scrape,A Luxury Home in Los Angeles,<b>The space</b><br />Private 16 x 15 ft room ...,NaN,https://a0.muscache.com/pictures/821696/64d8a6...,61864,...,NaN,NaN,NaN,NaN,f,2,0,2,0,NaN
1,206662,https://www.airbnb.com/rooms/206662,20221206172243,2022-12-07,city scrape,"Hollywood & Hiking, 30 day minimum","Semi-Private, vaccinated only, you will be sta...",The quietest part of Hollywood yet still walka...,https://a0.muscache.com/pictures/30274679/5011...,33861,...,4.92,4.92,4.87,NaN,f,1,0,1,0,0.71
2,67089,https://www.airbnb.com/rooms/67089,20221206172243,2022-12-07,city scrape,Tranquil Guesthouse Oasis with a Hot Tub,Relax in a hammock or hot tub in a secluded ba...,The bungalow sits behind the main house down t...,https://a0.muscache.com/pictures/87cd0874-9acc...,210344,...,4.94,4.91,4.87,NaN,f,2,2,0,0,4.80
3,210489,https://www.airbnb.com/rooms/210489,20221206172243,2022-12-07,city scrape,3br 3ba townhome near Beverly Hills,"Beverly Hills area, very safe, quiet and centr...",Great neighborhood! Two blocks to Beverly Hil...,https://a0.muscache.com/pictures/7589741/7e3fc...,76111,...,4.93,4.43,4.64,Exempt - This listing is a transient occupancy...,f,2,1,1,0,0.14
4,109,https://www.airbnb.com/rooms/109,20221206172243,2022-12-07,city scrape,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...",NaN,https://a0.muscache.com/pictures/4321499/1da98...,521,...,4.00,5.00,4.00,NaN,f,1,1,0,0,0.01


In [ ]:
df_listing.shape

(40438, 75)

In [ ]:
#convert the price to float type
df["price_"] =df['price'].str.extract("(\d+)")

In [ ]:
df=df.dropna()

In [ ]:
df["price_"] = df["price_"].astype('float')

/var/folders/2f/jqnjgr2s3yzfyw8nbf3yw7nc0000gn/T/ipykernel_19697/3124420133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price_"] = df["price_"].astype('int')


In [ ]:
# aggregate by listid
df.groupby("listing_id")["price_"].mean()

listing_id
109                   115.328767
2708                  118.000000
2732                  179.000000
2864                   69.295890
3021                   71.553425
                         ...    
775269321569777349    136.715068
775274422954920121    225.000000
775274851040449567    140.000000
775305210424227347     97.000000
775316807460359720    500.000000
Name: price_, Length: 40430, dtype: float64

In [ ]:
# load pretrained_model
nlp=spacy.load("en_core_web_lg")

In [ ]:
#fillna
df_listing["description"] =df_listing["description"].fillna(" ")

In [ ]:
def extract_ner(sentence):
    # Process the input text with the NER component
    if sentence:
        doc = nlp(sentence)
        LOCs = []
        # Print the named entities and their labels
        for ent in doc.ents:
            if ent.label_ in ["FAC","LOC","ORG"]:
                print(ent.text)
                LOCs.append(ent.text)
        return LOCs

In [ ]:
df_listing["loc_description"] =df_listing["description"].map(extract_ner)

In [ ]:
import re

In [ ]:
# there are some block need to be removed.
def replace_br(locs):
    newlocs=[]
    for loc in locs:
        loc =re.sub('<br /><br /><b',"",loc)
        loc =re.sub('<br />',"",loc)
        loc =re.sub('<br /><br />',"",loc)
        loc =re.sub('<br',"",loc)
        loc =re.sub('/>',"",loc)
        
        newlocs.append(loc)
    return newlocs

In [ ]:
df_listing["loc_description_1"] = df_listing["loc_description"].map(replace_br)

In [ ]:
# explode the extraction so that we can agg by single LOC Or FAC
list_loc =df_listing[['id',"loc_description_1"]].explode("loc_description_1")

In [ ]:
list_loc_group =list_loc.groupby("loc_description_1").count()

In [ ]:
list_price = df[["listing_id",'price_']]

In [ ]:
list_price =list_price.groupby("listing_id").mean()

In [ ]:
loc_price =list_loc.merge(list_price,left_on="id",right_index=True)

In [ ]:
loc_price_group =loc_price.groupby('loc_description_1')['loc_description_1',"price_"].agg({"price_":'mean','loc_description_1':'count'})

/var/folders/2f/jqnjgr2s3yzfyw8nbf3yw7nc0000gn/T/ipykernel_19697/3921981899.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  loc_price_group =loc_price.groupby('loc_description_1')['loc_description_1',"price_"].agg({"price_":'mean','loc_description_1':'count'})


In [ ]:
loc_price_group.rename(columns={'loc_description_1':'count_loc'},inplace=True)

In [ ]:
top_200_loc_price_group=loc_price_group.sort_values('count_loc',ascending=False)[:100].sort_values('price_',ascending=False)

In [ ]:
#  loc is strongly correlated with price, especially the landmark.

In [ ]:
top_200_loc_price_group[:20]

,price_,count_loc
loc_description_1,,
Avalon,366.243581,129
Villa,300.816718,137
Pacific,284.540512,115
Catalina Island,281.057421,133
Ocean,275.540594,202
Mediterranean,273.493376,158
Backyard,264.927078,180
the Pacific Ocean,261.609455,204
Ventura Blvd,261.522963,304


In [ ]:
top_200_loc_price_group[20:40]

,price_,count_loc
loc_description_1,,
the Sunset Strip,224.707937,126
Silver Lake,224.395656,153
SoFi Stadium,219.073539,215
Rodeo Drive,216.589508,481
Roku TV,215.516147,160
CDC,214.805795,200
the Rose Bowl,213.375734,147
LA.,211.693249,140
Santa Monica Pier,209.325339,310


In [ ]:
# some name need to be unified

In [ ]:
def drop_stop(loc):
    new_loc = []
    for word in loc.split():
        if word not in ['The','the']:
            new_loc.append(word.lower().rstrip('s'))
    return " ".join(new_loc)


In [ ]:
list_loc_group =list_loc_group.reset_index()
list_loc_group["loc_description_1"] =list_loc_group["loc_description_1"].map(drop_stop)

In [ ]:
list_loc_group =list_loc_group.groupby('loc_description_1')["id"].sum().reset_index()

In [ ]:
list_loc["loc_description_1"] =list_loc["loc_description_1"].fillna(" ").map(drop_stop)

In [ ]:
loc_price =list_loc.merge(list_price,left_on="id",right_index=True)

In [ ]:
loc_price_group =loc_price.groupby('loc_description_1')['loc_description_1',"price_"].agg({"price_":'mean','loc_description_1':'count'})

/var/folders/2f/jqnjgr2s3yzfyw8nbf3yw7nc0000gn/T/ipykernel_19697/3921981899.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  loc_price_group =loc_price.groupby('loc_description_1')['loc_description_1',"price_"].agg({"price_":'mean','loc_description_1':'count'})


In [ ]:
loc_price_group

,price_,loc_description_1
loc_description_1,,
,175.793628,9231
& n,173.327397,2
cal mountain w ocean view sleeping,207.090411,1
harvard,299.000000,1
hopping center & racetrack,312.071233,1
...,...,...
🔆,75.000000,1
🛀🛁five,49.000000,1
🤎,215.986301,2


In [ ]:
loc_price_group.rename(columns={'loc_description_1':'count_loc'},inplace=True)

In [ ]:
top_200_loc_price_group=loc_price_group.sort_values('count_loc',ascending=False)[:100].sort_values('price_',ascending=False)

In [ ]:
top_200_loc_price_group[:20]

,price_,count_loc
loc_description_1,,
villa,317.472127,147
catalina island,291.248154,138
venice canal,277.833445,159
ocean,274.249845,217
santa monica mountain,273.101864,133
pacific ocean,268.676919,292
mediterranean,268.158074,165
backyard,266.765210,189
ventura blvd,261.048845,309


In [ ]:
top_200_loc_price_group[20:40]

,price_,count_loc
loc_description_1,,
south bay,227.589499,196
silver lake,223.243194,169
roku tv,217.677982,192
forum,217.289122,136
sofi,216.605496,162
rodeo drive,216.589508,481
cdc,214.805795,200
rose bowl,212.046721,244
la.,211.693249,140


In [ ]:
top_200_loc_price_group[40:60]

,price_,count_loc
loc_description_1,,
burbank airport,199.504685,138
griffith observatory,198.943300,184
amazon,198.681580,412
disneyland,198.459927,940
grove,191.668467,311
apple tv,190.023813,319
disney,189.682742,264
washer/dryer,189.399321,214
lax,188.316208,1816
